In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda beautifulsoup4 --yes
!conda install lxml --yes
import requests
import bs4
import lxml

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1  --> 2019.6.16-py36_1  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1c-h7b6447c_1 --> 1.1.1c-h516909a_0 conda-forge


certifi-2019.6.16    | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of cond

Introduction

Harry is thinking about opening a Pizza cuisine. He is currently considering 2 options between New York, where he can stay close to his old mom, and Toronto, his spouse's hometown. To make a decision, he ask me to get some figure about the competitors in 2 areas. This analysis will help guide Harry on how many competitors will be around if he decides to run his own Pizza shop.

Data Collection and Usage

In this assignment we will collect the venues around the cities from Foursquare service. Then we will find the cities where there is less number of pizza and Italian cuisine business to observe the potential prospect.

Get New York map

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Data downloaded!


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Defind Foursquare credential

In [19]:
CLIENT_ID = 'QJRGPIGUGTJ5DWZKPESXSFCMTDOIDBWF4RTCF0WHSNWV2NL3' # your Foursquare ID
CLIENT_SECRET = 'BUM2WUONMARC11MG2T3MWHVN3DGG5KKXTTLFNSBS1AICWVUV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJRGPIGUGTJ5DWZKPESXSFCMTDOIDBWF4RTCF0WHSNWV2NL3
CLIENT_SECRET:BUM2WUONMARC11MG2T3MWHVN3DGG5KKXTTLFNSBS1AICWVUV


In [5]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Done! shape is ', nearby_venues.shape)
    return(nearby_venues)

In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5d77a048e97dfb002c7d10e8'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 40.62181079702983,
    'lng': -74.07582271430262},
   'sw': {'lat': 40.612810788029826, 'lng': -74.08765712993663}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dadc12093a070582bb6332a',
       'name': 'SUBWAY',
       'location': {'lat': 40.618939,
        'lng': -74.082881,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.618939,
          'lng': -74.082881}],
        'distance': 205,
        'postalCode': '10304',
        'cc': 'US',
        'city': 'Staten Island',
        'state': 'NY',


In [21]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Done! shape is  (10412, 7)


In [33]:
NY_competitors = newyork_venues[['Venue Category', 'Venue']].loc[np.logical_or(newyork_venues['Venue Category'] == 'Pizza Place', newyork_venues['Venue Category'] == 'Italian Restaurant')].groupby(['Venue Category']).count()
NY_areal = 783.8
print('There are {} Pizza restaurants and {} Italian restaurants in New York.'.format(NY_competitors['Venue'].iloc[0], NY_competitors['Venue'].iloc[1]))
'New York covers {} sq.km. which yield {:0.2f} Pizza restautants per sq.km. and {:0.2f} Italian restautants per sq.km.'.format(NY_areal, NY_competitors['Venue'].iloc[0]/NY_areal, NY_competitors['Venue'].iloc[1]/NY_areal)

There are 318 Pizza restaurants and 446 Italian restaurants in New York.


'New York covers 783.8 sq.km. which yield 0.41 Pizza restautants per sq.km. and 0.57 Italian restautants per sq.km.'

In [23]:
# retreive response from provided page
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.read_html(response.text)
df_postcode = df[0]
df_postcode = df_postcode[df_postcode.Borough != 'Not assigned'] # remove row which 'Borough' is 'Not assigned'
df_postcode['Neighbourhood'].loc[df_postcode['Neighbourhood'] == 'Not assigned'] = df_postcode['Borough'] # copy 'Borough' value to 'Neighbourhood' column for 'Neighbourhood' whose value is 'Not assigned'
df_postcode = df_postcode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index() # group row with same 'Postcode' and 'Borough' and join 'Neighborhood' with ',' and reset index of dataframe
df_postcode = df_postcode[['Postcode', 'Borough', 'Neighbourhood']] # pick only required columns

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
# define latitude and longitude function
def lat_lng(postcode):
    return df_lat_lng.loc[df_lat_lng['Postal Code'] == postcode]

In [25]:
# download coordinates from csv file
df_lat_lng = pd.read_csv('Geospatial_Coordinates.csv')

df_postcode[['Latitude', 'Longitude']] = lat_lng(df_postcode['Postcode'])[['Latitude', 'Longitude']]
df_TR = df_postcode[df_postcode['Borough'].str.contains('Toronto')]

In [27]:
# find average lat, long of Toronto
TR_latitude = df_TR['Latitude'].mean()
TR_longitude = df_TR['Longitude'].mean()
print('{}, {}'.format(TR_latitude, TR_longitude))
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[TR_latitude , TR_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_TR['Latitude'], df_TR['Longitude'], df_TR['Borough'], df_TR['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

43.667262184210514, -79.38988323421053


In [30]:
TR_venues = getNearbyVenues(names=df_TR['Neighbourhood'],
                                   latitudes=df_TR['Latitude'],
                                   longitudes=df_TR['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [32]:
TR_competitors = TR_venues[['Venue Category', 'Venue']].loc[np.logical_or(TR_venues['Venue Category'] == 'Pizza Place', TR_venues['Venue Category'] == 'Italian Restaurant')].groupby(['Venue Category']).count()
TR_areal = 630.2
print('There are {} Pizza restaurants and {} Italian restaurants in Toronto'.format(TR_competitors['Venue'].iloc[0], TR_competitors['Venue'].iloc[1]))
'Toronto covers {} sq.km. which yield {:0.2f} Pizza restautants per sq.km. and {:0.2f} Italian restautants per sq.km.'.format(TR_areal, TR_competitors['Venue'].iloc[0]/TR_areal, TR_competitors['Venue'].iloc[1]/TR_areal)

There are 45 Pizza restaurants and 32 Italian restaurants in Toronto


'Toronto covers 630.2 sq.km. which yield 0.07 Pizza restautants per sq.km. and 0.05 Italian restautants per sq.km.'

the density of pizza and Italian restaurants are much lower in Toronto so it is a good idea to opt the location with less number of competitors. Thus, we should continue analysis in 

In [42]:
TR_competitors = TR_venues.loc[np.logical_or(TR_venues['Venue Category'] == 'Pizza Place', TR_venues['Venue Category'] == 'Italian Restaurant')].reset_index(drop=True)
TR_competitors.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
1,"The Danforth West, Riverdale",43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant
2,"The Danforth West, Riverdale",43.679557,-79.352188,Pizzeria Libretto,43.678489,-79.347576,Pizza Place
3,"The Danforth West, Riverdale",43.679557,-79.352188,Il Fornello,43.678604,-79.346904,Italian Restaurant
4,"The Beaches West, India Bazaar",43.668999,-79.315572,Casa di Giorgio,43.666645,-79.315204,Italian Restaurant


In [49]:
# create a map displaying all pizza and Italian restaurants in Toronto
map_competitors = folium.Map(location=[TR_latitude , TR_longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood, venue, cat in zip(TR_competitors['Venue Latitude'], TR_competitors['Venue Longitude'], TR_competitors['Neighborhood'], TR_competitors['Venue'], TR_competitors['Venue Category']):
    label = '{}, {} ({})'.format(venue, neighborhood, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_competitors)  
    
map_competitors

Cluster zone to observe area with smallest number of restaurants

In [51]:
# set number of clusters
kclusters = 5

TR_grouped_clustering = TR_competitors[['Venue Latitude','Venue Longitude']]

# run k-means clustering
TR_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TR_grouped_clustering)
TR_competitors['cluster_label'] = TR_kmeans.labels_
TR_competitors.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,cluster_label
0,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant,3
1,"The Danforth West, Riverdale",43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant,3
2,"The Danforth West, Riverdale",43.679557,-79.352188,Pizzeria Libretto,43.678489,-79.347576,Pizza Place,3
3,"The Danforth West, Riverdale",43.679557,-79.352188,Il Fornello,43.678604,-79.346904,Italian Restaurant,3
4,"The Beaches West, India Bazaar",43.668999,-79.315572,Casa di Giorgio,43.666645,-79.315204,Italian Restaurant,3


In [53]:
TR_competitors[['Venue', 'cluster_label']].groupby('cluster_label').count()

,Venue
cluster_label,
0,45
1,5
2,8
3,9
4,10


Discussion

To optimize the location for the restaurant, select cluster zone 1 which has the lowest number of competitors and gain the highest number of potential customers.

In [56]:
TR_competitors.loc[TR_competitors['cluster_label'] == 1].reset_index(drop=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,cluster_label
0,"High Park, The Junction South",43.661608,-79.464763,nodo,43.665303,-79.465621,Italian Restaurant,1
1,"Parkdale, Roncesvalles",43.648960,-79.456325,Domani Restaurant & Wine Bar,43.649235,-79.450229,Italian Restaurant,1
2,"Runnymede, Swansea",43.651571,-79.484450,Campo,43.655191,-79.487067,Italian Restaurant,1
3,"Runnymede, Swansea",43.651571,-79.484450,Goodfellas Wood Oven Pizza,43.648224,-79.486356,Italian Restaurant,1
4,"Runnymede, Swansea",43.651571,-79.484450,Classico,43.649194,-79.484699,Pizza Place,1


Conclusion

Potential locations to run a new pizza business are in High Park, The Junction South, Parkdale, Roncesvalles, Runnymede and Swansea areas.